In [10]:
import sys, os
sys.path.append(os.path.realpath(".."))

import util_funcs
from importlib import reload
reload(util_funcs)
from copy import deepcopy as cp

import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import pywt
import tsfresh.feature_extraction.feature_calculators as feats
import constants
import clinical_text_analysis as cta
reload(cta)
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, r2_score
from os import path

reload(read)
import wf_analysis.datasets as wfdata
import wf_analysis.filters as filt
reload(filt)
from addict import Dict
import time
import pyedflib as edf

In [11]:
file_dict = Dict()
#v1.1 of tuh_eeg data subset
util_funcs.read_config() #load basics in, then override
util_funcs.config["data_dir_root"] = "/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf"
util_funcs.config['train_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/train/01_tcp_ar/'
util_funcs.config['combined_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/combined/01_tcp_ar/'
util_funcs.config['dev_test_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/dev_test/01_tcp_ar/'
util_funcs.config['train_02_tcp_le'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/train/02_tcp_le/'
util_funcs.config['dev_test_02_tcp_le'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/dev_test/02_tcp_le/'
util_funcs.config['train_03_tcp_ar_a'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/dev_test/03_tcp_ar_a/'
util_funcs.config['dev_test_03_tcp_ar_a'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/train/03_tcp_ar_a/'

In [12]:
reload(read)
all_files_v1 = read.get_all_token_file_names(None, "01_tcp_ar")
all_sessions_v1 = read.get_session_dir_names(None, "01_tcp_ar")

In [13]:
# files = read.get_all_token_file_names(None, "01_tcp_ar")

In [14]:
# file_lens = pd.Series(index=files)

In [16]:
# with edf.EdfReader(files[0]) as r:
#     print(r.file_duration)

401


In [17]:
# file_lens = pd.Series(index=files)

In [18]:
# def get_fd(file):
#     with edf.EdfReader(file, check_file_size=edf.DO_NOT_CHECK_FILE_SIZE) as reader:
#             return file, reader.file_duration, reader.getSignalLabels()

In [19]:
# get_fd(files[0])

('/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/01_tcp_ar/011/00001180/s002_2012_05_02/00001180_s002_t001.edf',
 401,
 ['EEG FP1-REF',
  'EEG FP2-REF',
  'EEG F3-REF',
  'EEG F4-REF',
  'EEG C3-REF',
  'EEG C4-REF',
  'EEG P3-REF',
  'EEG P4-REF',
  'EEG O1-REF',
  'EEG O2-REF',
  'EEG F7-REF',
  'EEG F8-REF',
  'EEG T3-REF',
  'EEG T4-REF',
  'EEG T5-REF',
  'EEG T6-REF',
  'EEG A1-REF',
  'EEG A2-REF',
  'EEG FZ-REF',
  'EEG CZ-REF',
  'EEG PZ-REF',
  'EEG ROC-REF',
  'EEG LOC-REF',
  'EEG EKG1-REF',
  'EEG T1-REF',
  'EEG T2-REF',
  'PHOTIC-REF',
  'IBI',
  'BURSTS',
  'SUPPR'])

In [20]:
# def run_q(torun, toret):
#     for i in iter(torun.get, None):
#             toret.put(get_fd(i))
# from multiprocessing import Process, Manager
# m = Manager()
# toRun = m.Queue()
# toReturn = m.Queue()


In [21]:
# processes = [Process(target=run_q, args=(toRun, toReturn)) for i in range(20)]

In [22]:
# res = [toRun.put(file) for file in files]

In [23]:
# res = [toRun.put(None) for i in range(16)]

In [24]:
# [p.start() for p in processes]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [29]:
# toRun.qsize(), toReturn.qsize()

(25425, 9195)

In [30]:
# channelList = []
# fileList = []
# for i in range(toReturn.qsize()):
#     file, length, channels = toReturn.get()
#     file_lens[file] = length
#     fileList.append(file)
#     channelList.append(channels)

In [31]:
# commonChannels = set(channelList[0])
# for channels in channelList:
#     commonChannels = commonChannels.intersection(set(channels))

In [32]:
# len(commonChannels)

21

In [33]:
# len(channelList)

10639

In [35]:
# len(fileList)

10639

In [34]:
# common_subset = set(util_funcs.get_common_channel_names())

In [75]:
# count = 0
# for i in range(len(channelList)):
    
#     if len(common_subset.intersection(set(channelList[i]))) == 21:
#         count += 1
#     else:
#         print(fileList[i])

/n/scratch2/ms994/tuh_files/tuh_eeg/all//01_tcp_ar/117/00011700/s001_2014_06_11/00011700_s001_t000.edf
/n/scratch2/ms994/tuh_files/tuh_eeg/all//01_tcp_ar/114/00011488/s002_2014_06_06/00011488_s002_t001.edf


In [54]:
#v1.2 of tuh_eeg data subset
util_funcs.read_config()
util_funcs.config["data_dir_root"] = "/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf"
util_funcs.config['train_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/train/01_tcp_ar/'
util_funcs.config['combined_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/combined/01_tcp_ar/'
util_funcs.config['dev_test_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/dev_test/01_tcp_ar/'
util_funcs.config['train_02_tcp_le'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/train/02_tcp_le/'
util_funcs.config['dev_test_02_tcp_le'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/dev_test/02_tcp_le/'
util_funcs.config['train_03_tcp_ar_a'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/dev_test/03_tcp_ar_a/'
util_funcs.config['dev_test_03_tcp_ar_a'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/v1.2.0/edf/train/03_tcp_ar_a/'

In [55]:
reload(read)
all_files_v2 = read.get_all_token_file_names(None, "01_tcp_ar")
all_sessions_v2 = read.get_session_dir_names(None, "01_tcp_ar")

In [56]:
len(all_files_v1), len(all_files_v2)

(34625, 15727)

In [5]:
patient_session_tuples_v1=[]
for file in all_files_v1:
    _, patient, session, _ = read.parse_edf_token_path_structure(file)
    patient_session_tuples_v1.append(patient + session)
    
    
patient_session_tuples_v2=[]
for file in all_files_v2:
    _, patient, session, _ = read.parse_edf_token_path_structure(file)
    patient_session_tuples_v2.append(patient + session)

In [6]:
len(patient_session_tuples_v1), len(set(patient_session_tuples_v1))

(34623, 12905)

In [7]:
len(patient_session_tuples_v2), len(set(patient_session_tuples_v2))

NameError: name 'patient_session_tuples_v2' is not defined

In [ ]:
all_sess

In [8]:
for session_file in all_sessions_v1 + all_sessions_v2:
    _, patient_prefix, patient, session = read.parse_edf_token_path_structure(session_file)
    patient_prefix = "/n/scratch2/ms994/tuh_files/tuh_eeg/all/01_tcp_ar/" + patient_prefix
    root_path =  patient_prefix + "/" + patient
    if not os.path.exists(patient_prefix):
        os.mkdir(patient_prefix)
    if not os.path.exists(root_path):
        os.mkdir(root_path, )
    os.symlink(session_file, root_path + "/" +session)

FileExistsError: [Errno 17] File exists: '/n/scratch2/ms994/tuh_files/tuh_eeg/v1.1.0/edf/01_tcp_ar/011/00001180/s002_2012_05_02' -> '/n/scratch2/ms994/tuh_files/tuh_eeg/all/01_tcp_ar/011/00001180/s002_2012_05_02'

In [84]:
#v1.2 of tuh_eeg data subset
util_funcs.read_config()
util_funcs.config["data_dir_root"] = "/n/scratch2/ms994/tuh_files/tuh_eeg/all/"
util_funcs.config['train_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/train/01_tcp_ar/'
util_funcs.config['combined_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/combined/01_tcp_ar/'
util_funcs.config['dev_test_01_tcp_ar'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/dev_test/01_tcp_ar/'
util_funcs.config['train_02_tcp_le'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/train/02_tcp_le/'
util_funcs.config['dev_test_02_tcp_le'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/dev_test/02_tcp_le/'
util_funcs.config['train_03_tcp_ar_a'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/dev_test/03_tcp_ar_a/'
util_funcs.config['dev_test_03_tcp_ar_a'] ='/n/scratch2/ms994/tuh_files/tuh_eeg/all/train/03_tcp_ar_a/'

In [85]:
all_sessions = read.get_session_dir_names(None, "01_tcp_ar")

In [76]:
files = read.get_all_token_file_names(None, "01_tcp_ar")

In [3]:
file_lens = pd.Series(index=files)

In [4]:
files[0]

'/n/scratch2/ms994/tuh_files/tuh_eeg/all//01_tcp_ar/011/00001180/s002_2012_05_02/00001180_s002_t001.edf'

In [9]:
with edf.EdfReader(files[0]) as r:
    print(r.file_duration)

NameError: name 'files' is not defined

In [5]:
file_lens = pd.Series(index=files)

In [4]:
def get_fd(file):
    with edf.EdfReader(file, check_file_size=edf.DO_NOT_CHECK_FILE_SIZE) as reader:
            return file, reader.file_duration, reader.getSignalLabels()

In [8]:
get_fd(files[0])

('/n/scratch2/ms994/tuh_files/tuh_eeg/all//01_tcp_ar/011/00001180/s002_2012_05_02/00001180_s002_t001.edf',
 401)

In [77]:
def run_q(torun, toret):
    for i in iter(torun.get, None):
            toret.put(get_fd(i))
from multiprocessing import Process, Manager
m = Manager()
toRun = m.Queue()
toReturn = m.Queue()


In [78]:
processes = [Process(target=run_q, args=(toRun, toReturn)) for i in range(16)]

In [79]:
res = [toRun.put(file) for file in files]

In [80]:
res = [toRun.put(None) for i in range(16)]

In [81]:
[p.start() for p in processes]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [83]:
toRun.qsize(), toReturn.qsize()

(0, 50350)

In [84]:
channelList = []
fileList = []
for i in range(toReturn.qsize()):
    file, length, channels = toReturn.get()
    file_lens[file] = length
    fileList.append(file)
    channelList.append(channels)

In [85]:
commonChannels = set(channelList[0])
for channels in channelList:
    commonChannels = commonChannels.intersection(set(channels))

In [86]:
len(commonChannels)

21

In [87]:
len(channelList)

50350

In [31]:
common_subset = set(util_funcs.get_common_channel_names())

In [75]:
count = 0
for i in range(len(channelList)):
    
    if len(common_subset.intersection(set(channelList[i]))) == 21:
        count += 1
    else:
        print(fileList[i])

/n/scratch2/ms994/tuh_files/tuh_eeg/all//01_tcp_ar/117/00011700/s001_2014_06_11/00011700_s001_t000.edf
/n/scratch2/ms994/tuh_files/tuh_eeg/all//01_tcp_ar/114/00011488/s002_2014_06_06/00011488_s002_t001.edf


In [73]:
count

50350

In [74]:
len(channelList)

50352

In [34]:
set(channelList[0])

{'BURSTS',
 'EEG A1-REF',
 'EEG A2-REF',
 'EEG C3-REF',
 'EEG C4-REF',
 'EEG CZ-REF',
 'EEG EKG1-REF',
 'EEG F3-REF',
 'EEG F4-REF',
 'EEG F7-REF',
 'EEG F8-REF',
 'EEG FP1-REF',
 'EEG FP2-REF',
 'EEG FZ-REF',
 'EEG LOC-REF',
 'EEG O1-REF',
 'EEG O2-REF',
 'EEG P3-REF',
 'EEG P4-REF',
 'EEG PZ-REF',
 'EEG ROC-REF',
 'EEG T1-REF',
 'EEG T2-REF',
 'EEG T3-REF',
 'EEG T4-REF',
 'EEG T5-REF',
 'EEG T6-REF',
 'IBI',
 'PHOTIC-REF',
 'SUPPR'}

In [84]:
file_lens[file] = length
length

401

In [107]:
file_lens.isna().sum() / len(file_lens)

0.418235621226565

In [91]:
file_lens[files].to_csv("None_01_tcp_ar_file_lengths.csv")

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
